## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current_description,Lat,Lng,Hotel Name
0,0,Torbay,CA,70.57,broken clouds,47.6666,-52.7314,Gallows Cove
1,7,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,9,Nikolskoye,RU,67.78,few clouds,59.7035,30.7861,Tourist House - Sablino
3,13,Yellowknife,CA,72.28,broken clouds,62.4560,-114.3525,The Explorer Hotel
4,16,Hilo,US,80.26,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current_description</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]


In [4]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df.Country == "FR") & (vacation_df.City == 'Vertou'),:]
vacation_end = vacation_df.loc[(vacation_df.Country == "FR") & (vacation_df.City == 'Vertou'),:]
vacation_stop1 = vacation_df.loc[(vacation_df.Country == "FR") & (vacation_df.City == 'Auxerre'),:]
vacation_stop2 = vacation_df.loc[(vacation_df.Country == "FR") & (vacation_df.City == 'Chalons-En-Champagne'),:]
vacation_stop3 = vacation_df.loc[(vacation_df.Country == "FR") & (vacation_df.City == 'Paris'),:]
vacation_start

,City_ID,City,Country,Max Temp,Current_description,Lat,Lng,Hotel Name
234,546,Vertou,FR,67.98,scattered clouds,47.1687,-1.4693,Hotel La Louée


In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
#https://www.geeksforgeeks.org/python-convert-numpy-arrays-to-tuples/

start = tuple(map(tuple,(vacation_start[['Lat', 'Lng']].to_numpy())))[0]
end = tuple(map(tuple,(vacation_end[['Lat', 'Lng']].to_numpy())))[0]
stop1 = tuple(map(tuple,(vacation_stop1[['Lat', 'Lng']].to_numpy())))[0]
stop2 = tuple(map(tuple,(vacation_stop2[['Lat', 'Lng']].to_numpy())))[0]
stop3 = tuple(map(tuple,(vacation_stop3[['Lat', 'Lng']].to_numpy())))[0]
end

(47.1687, -1.4693)

In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
import gmaps
import gmaps.datasets
from config import g_key


fig = gmaps.figure()

Vertou2Paris=gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3], travel_mode='DRIVING')

fig.add_layer(Vertou2Paris)
fig



Figure(layout=FigureLayout(height='420px'))

In [8]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current_description,Lat,Lng,Hotel Name
0,546,Vertou,FR,67.98,scattered clouds,47.1687,-1.4693,Hotel La Louée
1,354,Auxerre,FR,69.66,clear sky,47.8000,3.5667,Hôtel Les Maréchaux
2,238,Chalons-En-Champagne,FR,72.18,scattered clouds,48.9539,4.3672,Hotel D'Angleterre
3,432,Paris,FR,72.10,clear sky,48.8534,2.3488,Hôtel Britannique - Paris Centre
4,546,Vertou,FR,67.98,scattered clouds,47.1687,-1.4693,Hotel La Louée


In [9]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current_description} and {Max Temp} °F </dd>
</dl>

"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]


In [10]:
# 11a. Add a marker layer for each city to the map.

fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=itinerary_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))